<a href="https://colab.research.google.com/github/casalazara/Machine-Learning-IELE/blob/master/Reto%204/Reto4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Inicio

In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd 
from keras.preprocessing import image
from keras.models import load_model
from google.colab import drive
from tensorflow.keras.applications import VGG16
import tensorflow as tf
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
os.chdir('../gdrive/MyDrive/reto4')

In [3]:
trdata = ImageDataGenerator(rescale=1./255.)
traindata = trdata.flow_from_directory(directory="seg_train",target_size=(224,224))
tsdata = ImageDataGenerator(rescale=1./255.,validation_split=0.5)
validdata = tsdata.flow_from_directory(directory="seg_test", target_size=(224,224),subset='validation')
testdata = tsdata.flow_from_directory(directory="seg_test", target_size=(224,224),subset='training')

Found 14034 images belonging to 6 classes.
Found 1498 images belonging to 6 classes.
Found 1502 images belonging to 6 classes.


# VGG-16
![Arquitectura](https://miro.medium.com/max/850/1*_Lg1i7wv1pLpzp2F4MLrvw.png)


## Callback

In [4]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, weights_dir, patience=10, rate=0.5):
        """
        Initializer Method
        :param weights_dir: The directory of the model weights
        :param patience: Number of epochs before reducing lr
        :param rate: The rate to reduce the lr
        """
        super(CustomCallback, self).__init__()
        self.weights_dir = weights_dir
        self.rate = rate
        self.patience = patience

    def on_train_begin(self, logs=None):
        """
        Configurations to be taken an account on
        before training begins.
        :param logs: The logs from the output.
        """
        self.best_loss = np.Inf
        self.best_acc = 0
        self.wait = 0
        self.best_weights = None
    
    def on_epoch_end(self, epoch, logs=None):
        """
        Method that is called each time an epoch ends
        excecution
        :param epoch: The current epoch of the training
        :param logs: The logs from the output from an epoch ended
        """
        # Current validation loss
        current_loss = logs.get("val_loss")
        # Current validation Accuracy
        current_acc = logs.get("val_acc")
        if (current_loss < self.best_loss and 
            current_acc > self.best_acc):
            # We save the model weights
            # We wait again
            # We assign the best loss and the best accuracy.
            self.wait = 0
            self.best_loss = current_loss
            self.best_acc = current_acc
            self.model.save_weights(self.weights_dir)
            self.best_weights = self.model.get_weights()
            print("\nBest Weights Saved!!")
        else:
            self.wait += 1
            if self.wait >= self.patience:
                # Reduce learning rate by the initialized rate.
                self.wait = 0
                print(f"\nEpoch {epoch}, Reducing Learning Rate")
                # We set the new learning rate
                lr = K.get_value(self.model.optimizer.lr)
                new_lr = lr * self.rate
                K.set_value(self.model.optimizer.lr, new_lr)
                print(f"\nLearning Rate Reduced: {new_lr}")
                # We load the best weights to continue the training.
                self.model.set_weights(self.best_weights)
                print("\nBest Weights Loaded!!")

## Con Transfer Learning

### Carga del modelo

In [5]:
model=VGG16(include_top=False,input_shape=(224,224,3))

### Congelamiento de capas

In [6]:
for layer in model.layers:
    layer.trainable=False

### Últimas capas

In [7]:
output=model.layers[-1].output
model_final=Flatten()(output)
model_final=Dense(4096,activation='relu')(model_final)
model_final=Dense(4096,activation='relu')(model_final)
model_final=Dense(1000,activation='relu')(model_final)
model_final=Dense(6,activation='softmax')(model_final)

### Modelo final

In [8]:
model=tf.keras.models.Model(model.input,model_final)

### Compilación

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc','Precision','Recall'])

### Resumen del modelo

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Carga de pesos

Este código carga los pesos conseguidos para probarlos posteriormente

In [11]:
model.load_weights('train_best_weights_2.h5')

In [12]:
val_loss, val_acc,val_pre,val_rec = model.evaluate_generator(validdata,verbose=0)
print(f'Validation loss: {val_loss}, Validation acc: {val_acc}, Validation Precision: {val_pre}, Validation Recall: {val_rec}')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Validation loss: 0.331777423620224, Validation acc: 0.8931909203529358, Validation Precision: 0.902188777923584, Validation Recall: 0.8805073499679565


### Entrenamiento

In [ ]:
history = model.fit_generator(generator=traindata, validation_data= validdata, 
                              epochs=6,
                              callbacks=[CustomCallback('train_best_weights_3.h5')])

### Evaluación

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### Guardado del modelo y pesos

In [ ]:
model.save_weights('latest_weights_3.h5')
model.save('model_3.h5')

### Evaluación conjunto de prueba

In [13]:
test_loss, test_acc,test_pre,test_rec = model.evaluate_generator(testdata,verbose=0)
print(f'Test loss: {test_loss}, Test acc: {test_acc}, Test Precision: {test_pre}, Test Recall: {test_rec}')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Test loss: 0.3673647940158844, Test acc: 0.8801597952842712, Test Precision: 0.8896457552909851, Test Recall: 0.8695073127746582
